In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import product
os.chdir("../")
from src.data.processing_data import Format
from src.models.pipelines import Pipeline

In [ ]:
pd.set_option('display.max_columns', None)
sns.set_theme()

# Prediction Global Report

In [ ]:
_list_datasets_T = [["swda", 50],
                    ["dyda_da", 5],
                    ["mrda", 50],
                    ["dyda_e", 5],
                    ["meld_s", 5],
                    ["meld_e", 5]]

_list_encoder = ["bert-base-cased", "xlnet-base-cased"]

_list_data_format_decoder = [["MLP", "concatenate"], ["GRU", "separate"]]

_list_inputs = list(product(_list_datasets_T, _list_encoder, _list_data_format_decoder))

_possible_pipelines = {
                            "inputs": _list_inputs, 
                            "outputs": []
                        }

df_global_report = pd.DataFrame()

In [ ]:
_possible_pipelines

In [ ]:
for _input in _possible_pipelines["inputs"]:
    _pipeline = Pipeline(_input[0][0], _input[0][1], _input[1], _input[2][0], _input[2][1])
    _pipeline.summarize()
    df_global_report = pd.concat([df_global_report, _pipeline.df_report]).reset_index(drop=True)
    _possible_pipelines["outputs"].append(_pipeline.confusion_matrix)
    df_cm = pd.DataFrame(
    _pipeline.confusion_matrix,
    Format(_input[0][0], _input[0][1], "separate").get_distincts_labels(),
    Format(_input[0][0], _input[0][1], "separate").get_distincts_labels())
    plt.figure(figsize=(5, 5))
    sns.set(font_scale=1)
    sns.heatmap(df_cm,
                annot=True,
                annot_kws={"size": 8},
                cmap=sns.color_palette("ch:start=.2,rot=-.3", as_cmap=True), 
                cbar=False)
    plt.savefig(
        f"../intent_classification/figs/{_input[0][0]}_{_input[1].split("-")[0]}_{_input[2][0]}.pdf")
    plt.show()

In [ ]:
df_global_report

In [ ]:
_possible_pipelines["outputs"]